<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P3_prompt_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#以生成式对话机器人为例

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install datasets
!pip install accelerate==0.22.0
!pip install transformers==4.33.1
!pip install peft==0.5.0
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer,pipeline
from datasets import Dataset,load_dataset
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit, PeftModel
ds = load_dataset("zhengr/alpaca-chinese-dataset",cache_dir="./")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def data_process(example): #这次数据处理不进行batched，只处理单个的数据，因为label部分不太容易做
  max_length=256
  tokenized_input=tokenizer("\n".join(["User:"+example["instruction"],example["input"]]).strip()+"\nAssistant:")
  tokenized_output=tokenizer(example["output"]+tokenizer.eos_token)
  input_ids=tokenized_input["input_ids"]+tokenized_output["input_ids"]
  attention_mask=tokenized_input["attention_mask"]+tokenized_output["attention_mask"]
  labels= [-100]*len(tokenized_input["input_ids"])+tokenized_output["input_ids"]
  if len(input_ids)>max_length:
    input_ids=input_ids[:max_length]
    attention_mask=attention_mask[:max_length]
    labels=labels[:max_length]
  return {
      "input_ids":input_ids,
      "attention_mask":attention_mask,
      "labels":labels
  }
tokenized_ds=ds.map(data_process,remove_columns=ds["train"].column_names)

In [5]:
# Soft Prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10 #prompt token的最长限制)
# config
# Hard Prompt
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是你与user的对话内容，根据user的问题提供合理的回答。",
                            num_virtual_tokens=len(tokenizer("下面是你与user的对话内容，根据user的问题提供合理的回答。")["input_ids"]),
                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")
config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=15, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='下面是你与user的对话内容，根据user的问题提供合理的回答。', tokenizer_name_or_path='Langboat/bloom-1b4-zh')

In [6]:
model = get_peft_model(model, config)

In [7]:
model.print_trainable_parameters()

trainable params: 30,720 || all params: 1,303,142,400 || trainable%: 0.0023573785950023575


In [8]:
args = TrainingArguments(
    output_dir="./peft_gen_chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=10
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.039200


KeyboardInterrupt: 

In [10]:
model.device

device(type='cuda', index=0)

In [11]:
# model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
peft_model = PeftModel.from_pretrained(model=model, model_id="./peft_gen_chatbot/checkpoint-10/").to('cuda:0')
ipt = tokenizer("User:{}\n{}".format("心情不好可以做些什么缓解一下？", "").strip() + "\nAssistant:", return_tensors="pt").to('cuda:0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
peft_model.generate(**ipt, max_length=256, do_sample=True)

In [ ]:
tokenizer.decode(peft_model.generate(**ipt, max_length=256, do_sample=True)[0], skip_special_tokens=True)

In [ ]:
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0])
